In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from algorithm import (
                        NoRegularizationTrainer,
                        GradientTrainer,
                        EGTrainer,
                        CSVDataLoader,
                        MultiTaskModel,
                        MultiTaskDataset,
                        mlt_train_test_split,
                        # true_values_from_data_loader,
                        unique_value_counts,
                        Cindex,
                        brier_score,
                        )
import easydict
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

# Preparing Data

In [5]:
data = pd.read_csv('Dataset/processdata.csv', encoding='latin-1')
date_columns = ['Date.of.Last.Contact', 'Date.of.Diagnostic']
data[date_columns] = data[date_columns].apply(pd.to_datetime, errors='coerce')
data['Survival_Time'] = (data['Date.of.Last.Contact'] - data['Date.of.Diagnostic']).dt.days
data.loc[:, 'Survival_Time'] = data['Survival_Time'].replace({-1: 0})
data['indicater'] = np.where(data['Date.of.Death'].isna(), 0, 1)
# Staring time from 1
# data['Survival_Time'] = data['Survival_Time'] + 1

# Removing instances with survival time = 0
data = data.drop(data[data['Survival_Time'] == 0].index)


columns_to_drop = ['Date.of.Death', 'Date.of.Last.Contact', 'Date.of.Diagnostic',
                   'Type.of.Death',  # High Correlation with 'Date.of.Death' and events
                   ]
data.drop(columns=columns_to_drop, inplace=True)
print(data.isna().sum())
data.dropna(inplace=True)

train_indices, test_indices = train_test_split(
        range(len(data)), stratify=data['indicater'], random_state=1, test_size=0.25
    )




columns_to_one_hot = ['RCBP.Name', 'Raca.Color', 'State.Civil', 'Code.Profession', 'Name.Occupation',
                              'Status.Address',
                              'City.Address', 'Description.of.Topography', 'Topography.Code', 'Morphology.Description',
                              'Code.of.Morphology', 'Description.of.Disease', 'Illness.Code', 'Diagnostic.means',
                              'Extension',
                              #'Type.of.Death' # High Correlation with 'Date.of.Death' and events
                      ]

# Replace other values that are not in top 9, into "other"
for column in columns_to_one_hot:
    top_9_values = data[column].value_counts().nlargest(9).index
    data[column] = data[column].where(data[column].isin(top_9_values), 'other')

data = pd.get_dummies(data, columns=columns_to_one_hot)

X = data.drop(['Survival_Time', 'indicater'], axis=1)
time_all = data['Survival_Time'].values
event_all = data['indicater'].values

X_train = X.iloc[train_indices].copy()
X_test = X.iloc[test_indices].copy()
time_train = time_all[train_indices].copy()
time_test = time_all[test_indices].copy()
event_train = event_all[train_indices].copy()
event_test = event_all[test_indices].copy()

print((X_test.columns == X_train.columns).all())

columns_to_binarize = ['Gender', 'Indicator.of.Rare.Case']    
for column in columns_to_binarize:
    lb = LabelBinarizer()
    X_train[column] = lb.fit_transform(X_train[column])
    X_test[column] = lb.transform(X_test[column])

scaler = MinMaxScaler()
X_train['Age'] = scaler.fit_transform(X_train[['Age']])
X_test['Age'] = scaler.transform(X_test[['Age']])

Tmax = time_train.max()
num_intervals=7
intervals = [(i * (Tmax // num_intervals), (i + 1) * (Tmax // num_intervals)) for i in range(num_intervals)]

Y_train = np.zeros((len(time_train), num_intervals), dtype=np.int_)
# Until the event happens, value is 1. after that, it is 0
for i, time_val in enumerate(time_train):
    for j, (left, right) in enumerate(intervals):
        if time_val > right or (left < time_val <= right):
            Y_train[i, j] = 1
Y_train = torch.Tensor(Y_train)


Y_test = np.zeros((len(time_test), num_intervals), dtype=np.int_)
# Until the event happens, value is 1. after that, it is 0
for i, time_val in enumerate(time_test):
    for j, (left, right) in enumerate(intervals):
        if time_val > right or (left < time_val <= right):
            Y_test[i, j] = 1
Y_test = torch.Tensor(Y_test)



# Creat mask matrix
W_train = np.zeros((len(time_train), num_intervals), dtype=np.int_)
# Until the time we know he was alive the value is 1, after that is 0
for i, (time_val, event_val) in enumerate(zip(time_train, event_train)):
    for j, (left, right) in enumerate(intervals):
        if event_val == 0 and time_val < left:
            W_train[i, j] = 0
        else:
            W_train[i, j] = 1
W_train = torch.Tensor(W_train)



# Creat mask matrix
W_test = np.zeros((len(time_test), num_intervals), dtype=np.int_)
# Until the time we know he was alive the value is 1, after that is 0
for i, (time_val, event_val) in enumerate(zip(time_test, event_test)):
    for j, (left, right) in enumerate(intervals):
        if event_val == 0 and time_val < left:
            W_test[i, j] = 0
        else:
            W_test[i, j] = 1
W_test = torch.Tensor(W_test)


for col in X_train.columns:
    if X_train[col].dtype == bool:
        X_train[col] = X_train[col].astype(int)
X_train = torch.tensor(X_train.values, dtype=torch.float32)

for col in X_test.columns:
    if X_test[col].dtype == bool:
        X_test[col] = X_test[col].astype(int)
X_test = torch.tensor(X_test.values, dtype=torch.float32)

# Y_train = Y[train_indices]
# Y_test = Y[test_indices]
# W_train = W[train_indices]
# W_test  = W[test_indices]
Y_train_transform = [Y_train[:, i:i + 1] for i in range(Y_train.size(1))]
Y_test_transform = [Y_test[:, i:i + 1] for i in range(Y_test.size(1))]
W_train_transform = [W_train[:, i:i+1] for i in range(W_train.size(1))]
W_test_transform = [W_test[:, i:i+1] for i in range(W_test.size(1))]

print((W_test_transform[-1] == 0).any())

args = easydict.EasyDict({
    "batch_size": 64,
    "lr": 0.01,
    "epochs": 200,
    "clip": 5.0,
    "lambda_reg": 0.01,
    "save_path": "outputfiles",
    "eg_k" : 1, 
    "early_stop_patience":15,
})

train_dataset = MultiTaskDataset(X_train, Y_train_transform, W_train_transform, event_train)
test_dataset = MultiTaskDataset(X_test, Y_test_transform, W_test_transform, event_test)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, drop_last=True)


RCBP.Name                       0
Gender                          0
Age                             0
Raca.Color                    493
State.Civil                  6538
Code.Profession              1060
Name.Occupation              1060
Status.Address                  0
City.Address                    0
Description.of.Topography       0
Topography.Code                 0
Morphology.Description          0
Code.of.Morphology              0
Description.of.Disease          0
Illness.Code                    0
Indicator.of.Rare.Case          0
Diagnostic.means               40
Extension                       0
Survival_Time                   0
indicater                       0
dtype: int64
True
tensor(True)


In [16]:
class EditedCindexOptimized(torch.nn.Module):

    def __init__(self):

        super(EditedCindexOptimized, self).__init__()

    def forward(self, y, y_hat, status):
        if not torch.is_tensor(y):
            y = torch.tensor(y, dtype=torch.float32)
        if not torch.is_tensor(y_hat):
            y_hat = torch.tensor(y_hat, dtype=torch.float32)
        if not torch.is_tensor(status):
            status = torch.tensor(status, dtype=torch.float32)

        # replacing loop acceleration with matrix calculation
        
        y_diff = y.unsqueeze(1) - y.unsqueeze(0)
        y_hat_diff = y_hat.unsqueeze(1) - y_hat.unsqueeze(0)
        # status[i] and status[j] mark whether to censored data
        status_i = status.unsqueeze(1)
        status_j = status.unsqueeze(0)
        valid_pairs = torch.logical_or((y_diff < 0) & (status_i == 1), (y_diff > 0) & (status_j == 1)).float()
        torch.diagonal(valid_pairs).fill_(0) #Diagonal set to 0 to eliminate interference
        concordant_pairs = torch.logical_or((y_diff < 0) & (y_hat_diff < 0)&(status_i == 1),(y_diff > 0) & (y_hat_diff > 0)& (status_j == 1)).float()
        torch.diagonal(concordant_pairs).fill_(0) #Diagonal set to 0 to eliminate interference
        concordant_pairs = concordant_pairs.float()
        c_index = concordant_pairs.sum() / valid_pairs.sum()
        return c_index.item()
    

class OriginalCindexOptimized(torch.nn.Module):

    def __init__(self):

        super(OriginalCindexOptimized, self).__init__()

    def forward(self, y, y_hat, status):
        if not torch.is_tensor(y):
            y = torch.tensor(y, dtype=torch.float32)
        if not torch.is_tensor(y_hat):
            y_hat = torch.tensor(y_hat, dtype=torch.float32)
        if not torch.is_tensor(status):
            status = torch.tensor(status, dtype=torch.float32)

        # replacing loop acceleration with matrix calculation
        
        y_diff = y.unsqueeze(1) - y.unsqueeze(0)
        y_hat_diff = y_hat.unsqueeze(1) - y_hat.unsqueeze(0)
        # status[i] and status[j] mark whether to censored data
        status_i = status.unsqueeze(1)
        status_j = status.unsqueeze(0)
        valid_pairs = torch.logical_or((y_diff <= 0) & (status_i == 1), (y_diff >= 0) & (status_j == 1)).float()
        torch.diagonal(valid_pairs).fill_(0) #Diagonal set to 0 to eliminate interference
        concordant_pairs = torch.logical_or((y_diff <= 0) & (y_hat_diff <= 0)&(status_i == 1),(y_diff >= 0) & (y_hat_diff >= 0)& (status_j == 1)).float()
        torch.diagonal(concordant_pairs).fill_(0) #Diagonal set to 0 to eliminate interference
        concordant_pairs = concordant_pairs.float()
        c_index = concordant_pairs.sum() / valid_pairs.sum()
        return c_index.item()

# Results for Model with No regularization

In [7]:

model = MultiTaskModel(X_train.shape[1], Y_train.shape[1])
trainer = NoRegularizationTrainer(model,train_loader,test_loader, args)
trainer.train()


  0%|          | 1/200 [00:01<06:02,  1.82s/it]

End of Epoch 0, Average Training Loss: 3.0994, Average Gradient Norm: 0.0000
End of Epoch 0, Average Validation Loss: 2.4092
Current Learning Rate: 0.010000
End of Epoch 1, Average Training Loss: 2.5994, Average Gradient Norm: 0.0000


  1%|          | 2/200 [00:02<02:52,  1.15it/s]

End of Epoch 1, Average Validation Loss: 2.5157
Current Learning Rate: 0.010000
End of Epoch 2, Average Training Loss: 2.4805, Average Gradient Norm: 0.0000
End of Epoch 2, Average Validation Loss: 2.4121
Current Learning Rate: 0.010000


  2%|▏         | 4/200 [00:02<01:24,  2.33it/s]

...

End of Epoch 20, Average Validation Loss: 2.3904
Current Learning Rate: 0.000100


 11%|█         | 22/200 [00:06<00:46,  3.79it/s]

End of Epoch 21, Average Training Loss: 2.0645, Average Gradient Norm: 0.0000
End of Epoch 21, Average Validation Loss: 2.4035
Current Learning Rate: 0.000100


 11%|█         | 22/200 [00:06<00:55,  3.23it/s]

End of Epoch 22, Average Training Loss: 2.0443, Average Gradient Norm: 0.0000
End of Epoch 22, Average Validation Loss: 2.4032
Current Learning Rate: 0.000100
Early stopping triggered after 23 epochs.


## C-Index by edited verseion calculation

In [12]:



cindex_calculator_optimized = EditedCindexOptimized()

trainer.load_best_checkpoint()
predictions, Y_hat, Y_true, events = trainer.predict(train_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_train = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Training Data: {c11_train:.4f}")


predictions, Y_hat, Y_true, events = trainer.predict(test_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_test = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Test Data: {c11_test:.4f}")

Loaded model from checkpoint at epoch 8 with best validation loss 2.2558
C-index for Training Data: 0.7008
C-index for Test Data: 0.6873


## C-Index by the calculation you sent me

In [17]:



cindex_calculator_optimized = OriginalCindexOptimized()

trainer.load_best_checkpoint()
predictions, Y_hat, Y_true, events = trainer.predict(train_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_train = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Training Data: {c11_train:.4f}")


predictions, Y_hat, Y_true, events = trainer.predict(test_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_test = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Test Data: {c11_test:.4f}")

Loaded model from checkpoint at epoch 8 with best validation loss 2.2558
C-index for Training Data: 0.8738
C-index for Test Data: 0.8632


# Results for EG Based Regularization Model

In [18]:
model = MultiTaskModel(X_train.shape[1], Y_train.shape[1])
trainer = EGTrainer(model,train_loader,test_loader, train_dataset, args)
trainer.train()


  0%|          | 1/200 [00:00<02:39,  1.25it/s]

End of Epoch 0, Average Training Loss: 3.3657, Average Gradient Norm: 2.7779
End of Epoch 0, Average Validation Loss: 2.7763
Current Learning Rate: 0.010000


  1%|          | 2/200 [00:01<02:21,  1.39it/s]

End of Epoch 1, Average Training Loss: 2.9512, Average Gradient Norm: 2.0895
End of Epoch 1, Average Validation Loss: 2.8873
Current Learning Rate: 0.010000


  2%|▏         | 3/200 [00:02<02:21,  1.39it/s]

...

End of Epoch 22, Average Training Loss: 2.6997, Average Gradient Norm: 2.2107
End of Epoch 22, Average Validation Loss: 2.3913
Current Learning Rate: 0.000100


 12%|█▏        | 24/200 [00:17<02:10,  1.35it/s]

End of Epoch 23, Average Training Loss: 2.6563, Average Gradient Norm: 2.3533
End of Epoch 23, Average Validation Loss: 2.3991
Current Learning Rate: 0.000100


 12%|█▏        | 24/200 [00:18<02:14,  1.30it/s]

End of Epoch 24, Average Training Loss: 2.6498, Average Gradient Norm: 2.2652
End of Epoch 24, Average Validation Loss: 2.4004
Current Learning Rate: 0.000100
Early stopping triggered after 25 epochs.


## C-Index by edited verseion calculation

In [19]:

cindex_calculator_optimized = EditedCindexOptimized()
trainer.load_best_checkpoint()

predictions, Y_hat, Y_true, events = trainer.predict(train_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_train = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Training Data: {c11_train:.4f}")


predictions, Y_hat, Y_true, events = trainer.predict(test_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_test = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Test Data: {c11_test:.4f}")

Loaded model from checkpoint at epoch 10 with best validation loss 2.3783
C-index for Training Data: 0.6130
C-index for Test Data: 0.6075


## C-Index by the calculation you sent me

In [20]:
cindex_calculator_optimized = OriginalCindexOptimized()
trainer.load_best_checkpoint()

predictions, Y_hat, Y_true, events = trainer.predict(train_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_train = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Training Data: {c11_train:.4f}")


predictions, Y_hat, Y_true, events = trainer.predict(test_loader)
# Y_hat = predictions[np.arange(predictions.shape[0]), (Y_hat-1)]
c11_test = cindex_calculator_optimized(Y_true, Y_hat, events)
print(f"C-index for Test Data: {c11_test:.4f}")

Loaded model from checkpoint at epoch 10 with best validation loss 2.3783
C-index for Training Data: 0.8798
C-index for Test Data: 0.8772
